项目主要参考自：[PaddleX超简单之--【常规赛：PALM病理性近视预测】第二名方案](https://aistudio.baidu.com/aistudio/projectdetail/1889324?channelType=0&channel=0)

```
pip install paddlex -i https://mirror.baidu.com/pypi/simple
pip install paddle2onnx -i https://mirror.baidu.com/pypi/simple

In [1]:
# 划分训练集和测试集
import pandas as pd
import random

train_excel_file = 'dataset/Train/Classification.xlsx'
pd_list=pd.read_excel(train_excel_file)

pd_list_lenght=len(pd_list)
# 乱序
pd_list=pd_list.sample(frac=1)
offset=int(pd_list_lenght*0.9)
trian_list=pd_list[:offset]
eval_list=pd_list[offset:]
trian_list.to_csv("train_list.txt", index=None, header=None, sep=' ')
eval_list.to_csv("eval_list.txt", index=None, header=None, sep=' ')

In [3]:
# 设置使用0号GPU卡（如无GPU，执行此代码后仍然会使用CPU训练模型）
import matplotlib
matplotlib.use('Agg') 
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import paddlex as pdx

观察训练集，都是圆形的图片且亮点很多在左上或者左下。所以类似于sklearn.GridSearchCV的方法进行网格调参找水平翻转，垂直翻转，旋转角度的最优组合

```
{'transforms.RandomHorizontalFlip":[0.4,0.5,0.6]}
{'transforms.RandomVerticalFlip':[0.3,0.5,0.7]}
{'transforms.RandomRotate':[rotate_range=30,45,60,75]}

In [5]:
from paddlex.cls import transforms
train_transforms = transforms.Compose([

    transforms.RandomCrop(crop_size=224),
    transforms.RandomHorizontalFlip(0.6),
    transforms.RandomRotate(rotate_range=75),

    transforms.RandomVerticalFlip(),
    transforms.Normalize()
])
eval_transforms = transforms.Compose([
    transforms.ResizeByShort(short_size=256),
    transforms.CenterCrop(crop_size=224),
    transforms.Normalize()
])

In [6]:
train_dataset = pdx.datasets.ImageNet(
    data_dir='dataset/Train/fundus_image',
    file_list='train_list.txt',
    label_list='labels.txt',
    transforms=train_transforms,
    shuffle=True)
eval_dataset = pdx.datasets.ImageNet(
    data_dir='dataset/Train/fundus_image',
    file_list='eval_list.txt',
    label_list='labels.txt',
    transforms=eval_transforms)

2021-06-03 21:51:46 [INFO]	Starting to read file list from dataset...
2021-06-03 21:51:46 [INFO]	720 samples in file train_list.txt
2021-06-03 21:51:46 [INFO]	Starting to read file list from dataset...
2021-06-03 21:51:46 [INFO]	80 samples in file eval_list.txt


In [6]:
model = pdx.cls.ResNet50_vd_ssld(num_classes=2)
# model = pdx.cls.ResNet101_vd_ssld(num_classes=2)
model = pdx.cls.MobileNetV3_large_ssld(num_classes=2)
model.train(num_epochs=50,
            train_dataset=train_dataset,
            train_batch_size=128,
            eval_dataset=eval_dataset,
            lr_decay_epochs=[10, 20, 30],
            save_interval_epochs=5,
            learning_rate=0.025,
            save_dir='output/ResNet50_vd_ssld',
            use_vdl=True)

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/framework.py:689: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif dtype == np.bool:


2021-05-31 22:06:53 [INFO]	Downloading ResNet50_vd_ssld_pretrained.tar from https://paddle-imagenet-models-name.bj.bcebos.com/ResNet50_vd_ssld_pretrained.tar


  0%|          | 0/92837 [00:00<?, ?KB/s]100%|██████████| 92837/92837 [00:01<00:00, 51050.87KB/s]


2021-05-31 22:06:55 [INFO]	Decompressing output/ResNet50_vd_ssld/pretrain/ResNet50_vd_ssld_pretrained.tar...
2021-05-31 22:07:00 [INFO]	Load pretrain weights from output/ResNet50_vd_ssld/pretrain/ResNet50_vd_ssld_pretrained.
2021-05-31 22:07:00 [WARNING]	[SKIP] Shape of pretrained weight output/ResNet50_vd_ssld/pretrain/ResNet50_vd_ssld_pretrained/fc_0.w_0 doesn't match.(Pretrained: (2048, 1000), Actual: (2048, 2))
2021-05-31 22:07:00 [WARNING]	[SKIP] Shape of pretrained weight output/ResNet50_vd_ssld/pretrain/ResNet50_vd_ssld_pretrained/fc_0.b_0 doesn't match.(Pretrained: (1000,), Actual: (2,))
2021-05-31 22:07:01 [INFO]	There are 275 varaibles in output/ResNet50_vd_ssld/pretrain/ResNet50_vd_ssld_pretrained are loaded.
2021-05-31 22:07:25 [INFO]	[TRAIN] Epoch=1/50, Step=2/5, loss=0.63812, acc1=0.71875, acc2=1.0, lr=0.025, time_each_step=12.31s, eta=0:52:56
2021-05-31 22:07:45 [INFO]	[TRAIN] Epoch=1/50, Step=4/5, loss=0.4749, acc1=0.804688, acc2=1.0, lr=0.025, time_each_step=11.08s, et

100%|██████████| 1/1 [00:10<00:00, 10.67s/it]


...

2021-05-31 22:45:20 [INFO]	[EVAL] Finished, Epoch=40, acc1=0.975, acc2=1.0 .
2021-05-31 22:45:22 [INFO]	Model saved in output/ResNet50_vd_ssld/epoch_40.
2021-05-31 22:45:22 [INFO]	Current evaluated best model in eval_dataset is epoch_15, acc1=1.0
2021-05-31 22:45:43 [INFO]	[TRAIN] Epoch=41/50, Step=2/5, loss=0.044529, acc1=0.984375, acc2=1.0, lr=2.5e-05, time_each_step=9.72s, eta=0:8:10
2021-05-31 22:46:01 [INFO]	[TRAIN] Epoch=41/50, Step=4/5, loss=0.040087, acc1=0.984375, acc2=1.0, lr=2.5e-05, time_each_step=9.73s, eta=0:7:51
2021-05-31 22:46:15 [INFO]	[TRAIN] Epoch 41 finished, loss=0.04936, acc1=0.979688, acc2=1.0, lr=2.5e-05 .
2021-05-31 22:46:27 [INFO]	[TRAIN] Epoch=42/50, Step=1/5, loss=0.037626, acc1=0.984375, acc2=1.0, lr=2.5e-05, time_each_step=9.62s, eta=0:8:7
2021-05-31 22:46:43 [INFO]	[TRAIN] Epoch=42/50, Step=3/5, loss=0.04614, acc1=0.984375, acc2=1.0, lr=2.5e-05, time_each_step=9.51s, eta=0:7:48
2021-05-31 22:46:58 [INFO]	[TRAIN] Epoch=42/50, Step=5/5, loss=0.037091, acc1

100%|██████████| 1/1 [00:10<00:00, 10.04s/it]


2021-05-31 22:50:00 [INFO]	[EVAL] Finished, Epoch=45, acc1=0.975, acc2=1.0 .
2021-05-31 22:50:02 [INFO]	Model saved in output/ResNet50_vd_ssld/epoch_45.
2021-05-31 22:50:02 [INFO]	Current evaluated best model in eval_dataset is epoch_15, acc1=1.0
2021-05-31 22:50:14 [INFO]	[TRAIN] Epoch=46/50, Step=1/5, loss=0.030663, acc1=0.992188, acc2=1.0, lr=2.5e-05, time_each_step=9.77s, eta=0:4:36
2021-05-31 22:50:33 [INFO]	[TRAIN] Epoch=46/50, Step=3/5, loss=0.055259, acc1=0.960938, acc2=1.0, lr=2.5e-05, time_each_step=9.92s, eta=0:4:17
2021-05-31 22:50:50 [INFO]	[TRAIN] Epoch=46/50, Step=5/5, loss=0.029844, acc1=0.992188, acc2=1.0, lr=2.5e-05, time_each_step=10.0s, eta=0:3:57
2021-05-31 22:50:55 [INFO]	[TRAIN] Epoch 46 finished, loss=0.045759, acc1=0.982813, acc2=1.0, lr=2.5e-05 .
2021-05-31 22:51:18 [INFO]	[TRAIN] Epoch=47/50, Step=2/5, loss=0.023082, acc1=0.992188, acc2=1.0, lr=2.5e-05, time_each_step=10.05s, eta=0:3:21
2021-05-31 22:51:36 [INFO]	[TRAIN] Epoch=47/50, Step=4/5, loss=0.041724, 

100%|██████████| 1/1 [00:09<00:00,  9.03s/it]


2021-05-31 22:54:47 [INFO]	[EVAL] Finished, Epoch=50, acc1=0.975, acc2=1.0 .
2021-05-31 22:54:49 [INFO]	Model saved in output/ResNet50_vd_ssld/epoch_50.
2021-05-31 22:54:49 [INFO]	Current evaluated best model in eval_dataset is epoch_15, acc1=1.0


训练过程
![](https://ai-studio-static-online.cdn.bcebos.com/0f35f0467a8b4586a26cacaeb29bc3aec54d9ba6b27b4c868c23355922e34d14)


In [7]:
import paddlex as pdx
model = pdx.load_model('output/ResNet50_vd_ssld_new/best_model')
image_name = 'dataset/PALM-Testing400-Images/T0001.jpg'
result = model.predict(image_name, topk=2)
print("Predict Result:", result)

In [8]:
# 预测数据集val_list
val_list=[]
for i in range(1,401,1):
    filename='T'+ str(i).zfill(4)+'.jpg'
    print(filename)
    val_list.append(filename+'\n')

with open('val_list.txt','w') as f:
    f.writelines(val_list)
    
val_list=[]
with open('val_list.txt', 'r') as f:
    for line in f:
        line='dataset/PALM-Testing400-Images/'+line
        val_list.append(line.split('\n')[0])
        # print(line.split('\n')[0])
# print(val_list)

In [19]:

# 批量预测
import numpy as np
# model = pdx.load_model('output/ResNet50_vd_ssld/best_model')
model = pdx.load_model('best_model')
result = model.batch_predict(val_list)
print("Predict Result:", result)

2021-06-03 23:19:55 [INFO]	Model[ResNet50_vd_ssld] loaded.
Predict Result: [[{'category_id': 1, 'category': '病理性近视眼底彩照', 'score': 0.99996376}], [{'category_id': 0, 'category': '非病理性近视眼底彩照', 'score': 0.99999917}], [{'category_id': 0, 'category': '非病理性近视眼底彩照', 'score': 1.0}], [{'category_id': 1, 'category': '病理性近视眼底彩照', 'score': 0.999521}], [{'category_id': 1, 'category': '病理性近视眼底彩照', 'score': 0.9999963}], [{'category_id': 0, 'category': '非病理性近视眼底彩照', 'score': 0.99999976}], [{'category_id': 1, 'category': '病理性近视眼底彩照', 'score': 0.9941407}], [{'category_id': 1, 'category': '病理性近视眼底彩照', 'score': 0.9998398}], [{'category_id': 1, 'category': '病理性近视眼底彩照', 'score': 0.9992532}], [{'category_id': 1, 'category': '病理性近视眼底彩照', 'score': 0.9999285}], [{'category_id': 0, 'category': '非病理性近视眼底彩照', 'score': 0.9991247}], [{'category_id': 1, 'category': '病理性近视眼底彩照', 'score': 0.9998838}], [{'category_id': 1, 'category': '病理性近视眼底彩照', 'score': 0.9997199}], [{'category_id': 1, 'category': '病理性近视眼底彩照', 'score':

In [20]:
# 结果列
pd_B=[]
for item in result:
    # print(item)
    if item[0]['category_id']==1:
        pd_B.append(item[0]['score'])
    else:
        pd_B.append(1-item[0]['score'])

In [21]:
# 文件名列
pd_A=[]
with open('val_list.txt', 'r') as f:
    for line in f:
        pd_A.append(line.split('\n')[0])
        # print(line.split('\n')[0])

In [22]:
# 构造pandas的DataFrame
df= pd.DataFrame({'FileName': pd_A, 'PM Risk':pd_B})

In [23]:
# 保存为提交文件
df.to_csv("Classification_ResultsMay.csv", index=None)